In [36]:
import torch
import torchvision
import torch.nn.functional as F
import torch.nn
import matplotlib as plt
import pandas as pd


from torch.utils.data import DataLoader, TensorDataset, random_split
from torchvision.datasets.utils import download_url


# Download e esplorazione dei dati

In [37]:
DATASET_URL = "https://gist.github.com/BirajCoder/5f068dfe759c1ea6bdfce9535acdb72d/raw/c84d84e3c80f93be67f6c069cbdc0195ec36acbd/insurance.csv"
DATA_FILENAME = "insurance.csv"
download_url(DATASET_URL, '.')

Using downloaded and verified file: ./insurance.csv


In [38]:
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [39]:
your_name = 'Renato'

In [40]:
def customize_dataset(dataframe_raw, rand_str):

    dataframe = dataframe_raw.copy(deep = True)         # Crea una copia del dataframe, ponendo deep = True le modifiche apportate non modifica il dataframe originale

    # Eliminazione di alcune righe
    dataframe = dataframe.sample(int(0.95 * len(dataframe)), random_state = int(ord(rand_str[0])))
    # .sample(): Restituisce un numero specificato di righe    /    random_state(): controlla il mescolamento dei dati prima di dividerli

    # Scalatura input
    dataframe.bmi = dataframe.bmi * ord(rand_str[1])/100.

    # Scalatura target

    dataframe.charges = dataframe.charges * ord(rand_str[2])/100

    # Eliminazione colonne
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['region'], axis=1)

    return dataframe


In [41]:
dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()

,age,sex,bmi,children,smoker,charges
457,57,female,30.79995,0,no,13024.852555
1050,44,female,37.32455,1,no,8825.448995
56,58,female,32.14325,2,no,14968.105625
311,19,female,24.94700,0,no,1911.113600
1288,20,male,39.79400,2,yes,42179.022600


Domanda 1: quante righe ha **'dataset'**?

In [42]:
num_rows = len(dataframe.index)
num_rows

1271

Domanda 2: quante righe ha 'dataset'?

In [43]:
num_columns = len(dataframe.columns)
num_columns

6

Domanda 3: quali sono i titoli delle colonne?

In [44]:
input_cols = dataframe.columns.values
input_cols

array(['age', 'sex', 'bmi', 'children', 'smoker', 'charges'], dtype=object)

Domanda 4: Quali colonne non possiedono valori numerici?

In [60]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
categorical_cols = dataframe.select_dtypes(exclude = numerics).columns.values.tolist()
categorical_cols

['sex', 'smoker']